# TearSheet.ipynb

- doesn't actually generate a tearsheet like a classic print [S&P or Value Line tearsheet](https://financetrain.com/what-is-a-tear-sheet) on a company
- asks Perplexity a few basic questions about the company
- gets info from AlphaVantage
- opens a bunch of browser tabs from major services about the company
- could potentially save those and scrape them to make a tearsheet.


In [63]:
import dotenv
import sys
import os
import re
from datetime import datetime, timedelta
import time

import pandas as pd

import requests
import json

import openai
from openai import OpenAI

import IPython
from IPython.display import HTML, Image, Markdown, display

from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service


In [54]:
company = "Tesla"
symbol = "TSLA"


In [55]:
dotenv.load_dotenv()



True

In [3]:
system_prompt = """
You will act as a securities analyst and investment advisor with deep knowledge of financial markets, 
securities analysis, portfolio management. You will maintain a professional yet engaging tone, 
in the style of a senior investment bank research analyst."
"""

user_prompt = f"""You will focus on {company} ({symbol}), and provide a comprehensive analysis covering the following aspects:

Company Profile: An overview of {company}, including its lines of business, history, and recent key developments.

Major News: Significant events related to {company} or its industry impacting its stock.

Financial Performance: Recent earnings reports and stock performance compared to expectations, changes in dividends or stock buybacks.

Analyst Coverage: summarize recent changes to analysts' ratings noting which analyst and firms made upgrades or downgrades; summarize any recent short seller reports noting the firm and analyst.

Product Announcements: Launch of new products, strategic initiatives, or restructurings.

Strategic Moves: Information on deals, partnerships, mergers, acquisitions, divestitures, joint ventures, and major new business and revenue.

Securities Offerings: Announcements related to stock or bond issuances, buybacks, special dividends, or stock splits.

Management Changes: Significant personnel changes within {company}.

Stock Price Movements: Notable stock price changes and their reasons.

Timeline and Future Outlook: A timeline of these events """


In [4]:
perplexity_url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "pplx-7b-online",
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

perplexity_headers = {
    "Authorization": f"Bearer {os.getenv('PERPLEXITY_API_KEY')}",
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_str = response.json()['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
display(Markdown(response_str))


**Company Profile**

Tesla, Inc. is an American electric vehicle manufacturer and clean energy company headquartered in Austin, Texas. It was founded in 2003 by Elon Musk and Martin Eberhard. Tesla designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems worldwide. The company operates through two segments: Automotive and Energy Generation and Storage. Its products include electric vehicles such as sedans, sports utility vehicles, and semi-trucks, as well as solar energy products, battery energy storage from home to grid scale, and other related products and services.

**Recent Key Developments**

Tesla recently announced that it plans to raise prices for its Model Y vehicles due to increased costs associated with raw materials and logistics. Despite this, competitors like BYD, Li Auto, Nio, and Xpeng Motors continue to challenge Tesla in terms of value.

**Financial Performance**

In 2023, Tesla's revenue was \\$96.77 billion, representing an increase of 18.80% compared to the previous year's \\$81.46 billion. Earnings reached \\$15.00 billion, marking a rise of 19.44%. However, Tesla's stock price has faced challenges, with the stock dropping 29.25% in the first quarter, making it the worst performer on the S&P 500. Shares are now 57.5% below their late 2021 all-time high.

**Analyst Coverage**

As of the most recent data available, 35 analysts cover Tesla stock, with an average rating of "Hold". The 12-month stock price forecast is \\$205.07, representing an increase of 23.57% from the current price.

**Product Announcements**

Tesla has introduced new models and product offerings, such as the Cybertruck, which is expected to contribute significantly to the company's performance this year.

**Strategic Moves**

There have been no significant strategic moves or announcements from Tesla recently.

**Securities Offerings**

No notable securities offerings, buybacks, or dividends have been announced by Tesla.

**Management Changes**

There have been no significant management changes within Tesla.

**Stock Price Movements**

The stock price of Tesla has experienced volatility, with shares trading lower after the company missed analyst expectations for first-quarter deliveries.

**Future Outlook**

Looking ahead, Tesla faces challenges from both established automakers and emerging competitors in the electric vehicle space. The company must navigate these competitive pressures while continuing to innovate and grow its operations. Upcoming earnings reports will provide insights into Tesla's performance and future prospects.

In [5]:
user_prompt = f"""What are the current analyst ratings on {company} ({symbol})? 
Which short sellers issued reports on {company}, if any? Summarize any notable analyst reports."""

payload = {
    "model": "pplx-7b-online",
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_str = response.json()['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
display(Markdown(response_str))


As of March 2024, Tesla, Inc. (TSLA) has received mixed analyst ratings. According to Barchart.com, the average analyst rating for Tesla over the last three months was a moderate buy, with a score of 3.44 based on 27 analysts. Two months ago, the average rating was a hold, with a score of 3.28 based on 29 analysts. One month ago, the average rating remained a hold, with a score of 3.24 based on 29 analysts. Currently, the average rating is a hold, with a score of 3.10 based on 29 analysts.

In terms of specific analyst reports, there have been recent updates on Tesla from several firms. On March 25, 2024, Oppenheimer reiterated its hold rating on Tesla, suggesting investors to "buy the dip or bail?" due to pricing dynamics and expectations of lower prices before buying. However, on the same day, Vijay Rakesh from Mizuho Securities downgraded Tesla, along with Rivian and Nio, citing slowing electric vehicle (EV) demand, rising inventory levels, and pricing pressure. Additionally, Evercore ISI weighed in on Tesla's Model Y price hike, suggesting it could be a quarter-end marketing push rather than a significant shift in pricing strategy.

In [6]:
user_prompt = f"""Have there been any notable news stories about {company} ({symbol}) in the past year, including management profiles and investigative reports? 
For any notable stories, provide the date of publication and the media that reported them."""

payload = {
    "model": "pplx-7b-online",
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_str = response.json()['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
display(Markdown(response_str))


Yes, there have been several notable news stories about Tesla (TSLA) in the past year. Here are some examples:

1. **Investigative Report on Tesla's Management Team** - This article from Investopedia discusses the key members of Tesla's leadership team, including Elon Musk, Zachary Kirkhorn, Andrew Baglino, and others. It provides background information on their roles and responsibilities within the company.

2. **Tesla Probes Current and Former Executives** - Autonews reported that Tesla faced multiple investigations from regulatory bodies in the United States, including the National Highway Traffic Safety Administration (NHTSA).

3. **Allegations of Suppressing Complaints and Rigging Algorithms** - Reuters published an investigative report accusing Tesla of suppressing thousands of driving range complaints and rigging an algorithm to inflate its cars' in-dash range estimates, leading to a federal investigation.

4. **Tesla's Risk Profile** - GlobalData released a risk profile report on Tesla, highlighting the company's moderate risk profile, market position, revenue growth, and operational efficiency. It also discussed potential concerns related to low profitability and high debt levels.

5. **New Investigation into Several Popular Models** - The Street reported that the NHTSA opened a probe into several popular models of Tesla vehicles in response to a dozen complaints of reported loss of steering control.

In [44]:
# AlphaVantage overview
url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={os.environ['ALPHAVANTAGE_API_KEY']}"
r = requests.get(url)
data = r.json()
pd.DataFrame(data, index=[1]).transpose()


,1
Symbol,TSLA
AssetType,Common Stock
Name,Tesla Inc
Description,"Tesla, Inc. is an American electric vehicle an..."
CIK,1318605
Exchange,NASDAQ
Currency,USD
Country,USA
Sector,MANUFACTURING
Industry,MOTOR VEHICLES & PASSENGER CAR BODIES


In [46]:
exchange = data['Exchange']

morningstarmap = {'NYSE': 'xnys',
                  'NASDAQ': 'xnas'
                 }


In [105]:
now = datetime.now()
end_date = now.strftime('%Y%m%dT%H%M')
sdate = now - timedelta(days=2)
# sdate = datetime(now.year, now.month-1, now.day)
start_date = sdate.strftime('%Y%m%dT%H%M')
start_date, end_date

('20240401T1304', '20240403T1304')

In [107]:
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={os.environ["ALPHAVANTAGE_API_KEY"]}&time_from={start_date}&time_to={end_date}'

r = requests.get(url)
data = r.json()

for item in data['feed']:
    markdown_str = ""
    date_object = datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S")
    display_title = item['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item['summary'].replace("$", "\\\$")
    markdown_str += f"[{str(date_object)} {display_title}]({item['url']})\n {description}"
    display(Markdown(markdown_str))
    

[2024-04-03 16:10:57 Mark Cuban Predicts World's First 'Trillionaire' Won't Be Musk Or Bezos - It'll Be Someone Who 'Masters AI' And Applies It In Brand New Ways](https://www.benzinga.com/general/24/04/38073344/mark-cuban-predicts-worlds-first-trillionaire-wont-be-musk-or-bezos-itll-be-someone-who-masters-ai-a)
 A few years back, amid the fanfare surrounding tech titans like Elon Musk and Jeff Bezos, Mark Cuban made a forward-looking statement at SXSW Conference and Festivals in 2017. He declared that the world's first trillionaire would emerge not from the traditional tech sector but from someone ...

[2024-04-03 15:55:02 Ford Had a Different Start of 2024 Compared To General Motors And Even Tesla - Tesla  ( NASDAQ:TSLA ) ](https://www.benzinga.com/24/04/38072952/ford-had-a-different-start-of-2024-compared-to-general-motors-and-even-tesla)
 On Tuesday, the mighty EV king, Tesla Inc TSLA posted its first annual drop in sales since the first year of the pandemic. Simply put, rising EV competition from both Chinese, U.S. and European automakers ate into demand. General Motors GM posted a drop in deliveries as well as in EVs for the ...

[2024-04-03 15:53:40 Wall Street Wakes Up To The Risk In The Stock Market - Hoping For A Dovish Powell To Run Up Stocks - iShares 20+ Year Treasury Bond ETF  ( NASDAQ:TLT ) ](https://www.benzinga.com/24/04/38072931/wall-street-wakes-up-to-the-risk-in-the-stock-market-hoping-for-a-dovish-powell-to-run-up-stocks)
 To gain an edge, this is what you need to know today. Please click here for an enlarged version of the chart of iShares 20+ Year Treasury Bond ETF TLT. Focus on the downward sloping trendline on the chart. According to momo gurus, the reverse was supposed to happen.

[2024-04-03 15:51:01 Market Clubhouse Morning Memo - April 3rd, 2024  ( Trade Strategy For SPY, QQQ, AAPL, MSFT, NVDA, GOOGL, META, and TSLA )  - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) ](https://www.benzinga.com/24/04/38072858/market-clubhouse-morning-memo-april-3rd-2024-trade-strategy-for-spy-qqq-aapl-msft-nvda-googl-meta-an)
 Good Morning Traders! In today's Market Clubhouse Morning Memo, we will discuss SPY, QQQ, AAPL, MSFT, NVDA, GOOGL, META, and TSLA. Our proprietary formula, exclusive to Market Clubhouse, dictates these price levels. This dynamic equation takes into account price, volume, and options flow.

[2024-04-03 15:05:00 Tesla Stock Sinks After a Big Q1 Delivery Miss: ETFs in Focus](https://www.zacks.com/stock/news/2249881/tesla-stock-sinks-after-a-big-q1-delivery-miss-etfs-in-focus)
 Tesla Inc. (TSLA) disappointed investors with its first year-over-year drop in quarterly deliveries since 2020.

...

[2024-04-03 07:26:47 Dave & Buster's Entertainment  ( NASDAQ:PLAY )  Lowered to Sell at StockNews.com](https://www.defenseworld.net/2024/04/03/dave-busters-entertainment-nasdaqplay-lowered-to-sell-at-stocknews-com.html)
 StockNews.com downgraded shares of Dave & Buster's Entertainment ( NASDAQ:PLAY - Free Report ) from a hold rating to a sell rating in a research note issued to investors on Tuesday. Other equities research analysts also recently issued research reports about the company.

[2024-04-03 07:26:47 WNS  ( NYSE:WNS )  Lowered to Hold at StockNews.com](https://www.defenseworld.net/2024/04/03/wns-nysewns-lowered-to-hold-at-stocknews-com.html)
 StockNews.com downgraded shares of WNS ( NYSE:WNS - Free Report ) from a buy rating to a hold rating in a research note issued to investors on Tuesday.

[2024-04-03 07:26:47 Teradata  ( NYSE:TDC )  Downgraded by StockNews.com to Buy](https://www.defenseworld.net/2024/04/03/teradata-nysetdc-downgraded-by-stocknews-com-to-buy.html)
 StockNews.com cut shares of Teradata ( NYSE:TDC - Free Report ) from a strong-buy rating to a buy rating in a research report released on Tuesday morning.

[2024-04-03 07:26:47 Veeva Systems  ( NYSE:VEEV )  Upgraded by StockNews.com to Buy](https://www.defenseworld.net/2024/04/03/veeva-systems-nyseveev-upgraded-by-stocknews-com-to-buy.html)
 StockNews.com upgraded shares of Veeva Systems ( NYSE:VEEV - Free Report ) from a hold rating to a buy rating in a research note published on Tuesday. VEEV has been the subject of several other research reports.

[2024-04-03 07:26:46 NorthWestern Energy Group  ( NYSE:NWE )  Downgraded by StockNews.com to Sell](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html)
 StockNews.com lowered shares of NorthWestern Energy Group ( NYSE:NWE - Free Report ) from a hold rating to a sell rating in a research report released on Tuesday. Several other research analysts also recently weighed in on the company.

In [110]:
from newsapi import NewsApiClient
NEWSAPI_API_KEY = os.environ['NEWSAPI_API_KEY']
api = NewsApiClient(api_key=NEWSAPI_API_KEY)
sources = api.get_sources()
pd.DataFrame(sources['sources'])

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
1,abc-news-au,ABC News (AU),"Australia's most trusted source of local, nati...",https://www.abc.net.au/news,general,en,au
2,aftenposten,Aftenposten,Norges ledende nettavis med alltid oppdaterte ...,https://www.aftenposten.no,general,no,no
3,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",https://www.aljazeera.com,general,en,us
4,ansa,ANSA.it,"Agenzia ANSA: ultime notizie, foto, video e ap...",https://www.ansa.it,general,it,it
...,...,...,...,...,...,...,...
123,wired,Wired,"Wired is a monthly American magazine, publishe...",https://www.wired.com,technology,en,us
124,wired-de,Wired.de,Wired reports on how emerging technologies aff...,https://www.wired.de,technology,de,de
125,wirtschafts-woche,Wirtschafts Woche,Das Online-Portal des führenden Wirtschaftsmag...,http://www.wiwo.de,business,de,de
126,xinhua-net,Xinhua Net,"中国主要重点新闻网站,依托新华社遍布全球的采编网络,记者遍布世界100多个国家和地区,地方频...",http://xinhuanet.com/,general,zh,zh


In [119]:
# Calculate the date 24 hours ago
date_from = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S')

# The base URL for the NewsAPI
url = 'https://newsapi.org/v2/everything'

# Parameters for the API request
parameters = {
    'q': symbol,  # Query term
    'from': date_from,  # Get news from the last 24 hours
    'sortBy': 'publishedAt',  # Sort by publication date
    'apiKey': NEWSAPI_API_KEY  # Your API key
}

# Make the request to the NewsAPI
response = requests.get(url, params=parameters)

# Check if the request was successful
if response.status_code == 200:
    # Get the response data
    data = response.json()
    
    # Print the articles
    markdown_str = ""
    for article in data['articles']:
        date_object = datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S")
        display_title = article['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
        markdown_str += f"[{article['publishedAt']} {display_title}]({item['url']}) \n\n"

    display(Markdown(markdown_str))
else:
    print('Failed to retrieve data')

[2024-04-02T17:12:06Z Dow Jones Dives Nearly 500 Points; Cathie Wood Loads Up On This Stock Amid 45% Downtrend](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:48:30Z Top Stock Movers Now: Tesla, PVH, Humana, and More](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:22:02Z Tesla, UnitedHealth Group fall; Roivant, Valero rise, Tuesday, 4/2/2024](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:16:54Z Tesla reports its first year-over-year sales decline since the pandemic—and its shares are plummeting](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:10:43Z Tesla, Inc. (NASDAQ:TSLA) Shares Sold by Pallas Capital Advisors LLC](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:09:22Z BYD pode perder posto de maior do mundo em veículos elétricos para Tesla](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:07:37Z What You Missed On Wall Street This Morning](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T16:07:27Z BYD, Nio, Li and XPeng Report Deliveries Rose In March](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T15:28:05Z Tesla Stock Drops on Weak Delivery Numbers and it May Fall More](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T14:12:44Z Wall Street cai na abertura sob peso de seguradoras de saúde e Tesla](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T14:11:44Z Tesla Stock Tumbles as Q1 Deliveries Fall 8.5% From a Year Earlier](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T14:03:10Z Rivian Deliveries Grow 70% As The EV Startup Reaffirms 2024 Production Target](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T13:56:42Z Tesla shares drop 7% on first-quarter vehicle delivery and production report - CNBC](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T13:46:04Z Tesla misses first-quarter delivery estimate on slowing EV demand, tough competition](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T13:44:42Z Tesla stock slides following big Q1 delivery miss](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 

[2024-04-02T13:35:23Z Tesla Vehicle Production & Deliveries and Date for Financial Results & Webcast for First Quarter 2024 - Tesla Investor Relations](https://www.defenseworld.net/2024/04/03/northwestern-energy-group-nysenwe-downgraded-by-stocknews-com-to-sell.html) 



In [79]:
now = datetime.now()
end_date = now.strftime('%Y-%m-%d')
# start_date = datetime.now() - timedelta(days=30)
sdate = datetime(now.year, now.month-1, now.day)
start_date = sdate.strftime('%Y-%m-%d')
start_date, end_date

('2024-03-03', '2024-04-03')

In [94]:
API_KEY = os.environ['NEWSFILTER_API_KEY']
API_ENDPOINT = "https://api.newsfilter.io/search?token={}".format(API_KEY)

# Define the news search parameters
queryString = f"symbols:{symbol} AND publishedAt:[{start_date} TO {end_date}]"

payload = {
    "queryString": queryString,
    "from": 0,
    "size": 10
}

# Send the search query to the Search API
response = requests.post(API_ENDPOINT, json=payload)

# Read the response
articles = response.json()


# Read the response
articles = response.json()

for item in articles['articles']:
    markdown_str = ""
    date_array = item['publishedAt'].split("T")
    datestr = date_array[0] + "T" + date_array[1][:8]
#     print(item['publishedAt'])
#     print(date_array)
#     print(datestr)
    date_object = datetime.strptime(datestr, "%Y-%m-%dT%H:%M:%S")
    display_title = item['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item['description'].replace("$", "\\\$")

    markdown_str += f"[{str(date_object)} {display_title}]({item['url']})\n {description}"
    display(Markdown(markdown_str))

[2024-04-03 16:49:51 Baird Maintains Outperform on Tesla, Lowers Price Target to \\$280](https://newsfilter.io/articles/baird-maintains-outperform-on-tesla-lowers-price-target-to-280-70ffac859081da5a9fdfe3c5fe8ab934)
 Baird analyst Ben Kallo maintains Tesla with a Outperform and lowers the price target from $300 to $280.

[2024-04-03 13:36:16 Truist Securities Reiterates Hold on Tesla, Lowers Price Target to \\$176](https://newsfilter.io/articles/truist-securities-reiterates-hold-on-tesla-lowers-price-target-to-176-b7b2dfc1073fe46dd3848af9b7ba9280)
 Truist Securities analyst William Stein reiterates Tesla with a Hold and lowers the price target from $193 to $176.

[2024-04-03 13:11:15 JP Morgan Maintains Underweight on Tesla, Lowers Price Target to \\$115](https://newsfilter.io/articles/jp-morgan-maintains-underweight-on-tesla-lowers-price-target-to-115-35032a44204bb929d53f197699c5c28e)
 JP Morgan analyst Ryan Brinkman maintains Tesla with a Underweight and lowers the price target from $130 to $115.

[2024-04-03 12:59:00 What If Tesla Really Is Just Another Car Company? That’s a Nightmare for the Stock.](https://newsfilter.io/articles/what-if-tesla-really-is-just-another-car-company-thats-a-nightmare-for-the-stock-aee8a3440acbf69cc5d2f2a4f291b974)
 Tesla investors’ big nightmare might be that the electric-vehicle maker’s just an upscale Toyota Motor.

[2024-04-03 12:49:00 Intel, Tesla, SoFi, Cal-Maine, Dave & Buster’s, Disney, and More Stock Market Movers](https://newsfilter.io/articles/intel-tesla-sofi-cal-maine-dave--busters-disney-and-more-stock-market-movers-0d0c7bd7ef03b650fb9a23b92ed80d78)
 Intel stock tumbles after the company’s foundry business records an operating loss of about $7 billion in 2023, while Tesla extends losses after a sharp decline in first-quarter deliveries.

[2024-04-03 12:19:37 HSBC Maintains Reduce on Tesla, Lowers Price Target to \\$138](https://newsfilter.io/articles/hsbc-maintains-reduce-on-tesla-lowers-price-target-to-138-58ea27d95427b1bb030ee87e63767cee)
 HSBC analyst Michael Tyndall maintains Tesla with a Reduce and lowers the price target from $143 to $138.

[2024-04-03 12:18:00 Tesla Might Be Just Another Car Company. What That Means for the Stock.](https://newsfilter.io/articles/tesla-might-be-just-another-car-company-what-that-means-for-the-stock-37a55c132556baaebc9ad43a57a4bace)
 Tesla investors’ big nightmare might be that the electric-vehicle maker’s just an upscale Toyota Motor.

[2024-04-03 11:28:45 Noted Tesla bear says Musk's EV maker could 'go bust,' says stock is worth \\$14](https://newsfilter.io/articles/noted-tesla-bear-says-musks-ev-maker-could-go-bust-says-stock-is-worth-14-d9ca5a4df51dba5597dd5673f8352f49)
 Tesla is facing a price war, intense competition from Chinese players and weakening demand for its electric cars. 

[2024-04-03 10:38:00 Tesla’s Slump Exposes Magnificent 7 Cracks. Why Fed Rate Cuts Can Widen Them.](https://newsfilter.io/articles/teslas-slump-exposes-magnificent-7-cracks-why-fed-rate-cuts-can-widen-them-2e065ed5c17822ebd5f1b96d15a20a50)
 

[2024-04-03 08:51:00 Alibaba, AMD, Super Micro, Intel and Other Tech Stocks in Focus Today](https://newsfilter.io/articles/alibaba-amd-super-micro-intel-and-other-tech-stocks-in-focus-today-fb13643c43286112eb7dafa9e197830a)
 It’s been a tough start to the quarter for technology stocks. The tech-heavy Nasdaq Composite Index fell 1% Tuesday and looked set to open lower again Wednesday.

In [29]:
# open pages via selenium and firefox
outputdir = "htmldata"

# Print the formatted time
print(datetime.now().strftime('%H:%M:%S'), "Starting", flush=True)

firefox_app_path = '/Applications/Firefox.app'
# Path to your geckodriver
geckodriver_path = '/Users/drucev/webdrivers/geckodriver'

# Set up Firefox options to use your existing profile
# important for some sites that need a login, also a generic profile fingerprint that looks like a bot might get blocked
firefox_profile_path = '/Users/drucev/Library/Application Support/Firefox/Profiles/k8k0lcjj.default-release'
options = Options()
options.profile = firefox_profile_path

print(datetime.now().strftime('%H:%M:%S'), "Initialized profile", flush=True)

# Create a Service object with the path
service = Service(geckodriver_path)

print(datetime.now().strftime('%H:%M:%S'), "Initialized service", flush=True)
# Set up the Firefox driver
driver = webdriver.Firefox(service=service, options=options)

print(datetime.now().strftime('%H:%M:%S'), "Initialized webdriver", flush=True)
sleeptime = 10


16:49:12 Starting
16:49:46 Initialized profile
16:49:46 Initialized service
16:50:59 Initialized webdriver


In [30]:
# Open a new tab
def open_tab(driver, url):
    driver.execute_script(f"window.open('{url}');")
    # Switch to last one opened - sometimes hangs?
    # driver.switch_to.window(driver.window_handles[-1])
    # print(url)
    # time.sleep(sleeptime)  


In [31]:
url = f"https://stockcharts.com/h-sc/ui?s={symbol}&id=p33407302522&def=Y&listNum=1#"
source = "Stockcharts"
# Open the page
driver.get(url)

# Wait for the page to load
# time.sleep(sleeptime)  
display(Markdown(f"[{source}]({url})"))

[Stockcharts](https://stockcharts.com/h-sc/ui?s=TSLA&id=p33407302522&def=Y&listNum=1#)

In [32]:
url = f'https://www.bloomberg.com/search?query={company}'
source = "Bloomberg"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Bloomberg](https://www.bloomberg.com/search?query=Tesla)

In [33]:
url = f'https://www.reuters.com/site-search/?query={company}&offset=0'
source = "Reuters"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Reuters](https://www.reuters.com/site-search/?query=Tesla&offset=0)

In [34]:
url = f'https://finance.yahoo.com/quote/{symbol}/news'
source = "Yahoo quote"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Yahoo quote](https://finance.yahoo.com/quote/TSLA/news)

In [35]:
url = f'https://finance.yahoo.com/quote/{symbol}/key-statistics?ltr=1'
source = "Yahoo stats"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Yahoo stats](https://finance.yahoo.com/quote/TSLA/key-statistics?ltr=1)

In [36]:
url = f'https://www.ft.com/search?q={company}'
source = "FT"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[FT](https://www.ft.com/search?q=Tesla)

In [37]:
url = f'https://www.marketwatch.com/investing/stock/{symbol}?mod=search_symbol'
source = "Marketwatch"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Marketwatch](https://www.marketwatch.com/investing/stock/TSLA?mod=search_symbol)

In [38]:
url = f'https://www.barrons.com/market-data/stocks/{symbol}?mod=searchresults_companyquotes&mod=searchbar&search_keywords={company}&search_statement_type=suggested'    
source = "Barrons"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Barrons](https://www.barrons.com/market-data/stocks/TSLA?mod=searchresults_companyquotes&mod=searchbar&search_keywords=Tesla&search_statement_type=suggested)

In [39]:
url = f'https://www.businessinsider.com/s?q={company}'
source = "Insider"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Insider](https://www.businessinsider.com/s?q=Tesla)

In [40]:
url = f'https://www.google.com/finance/quote/{symbol}:{exchange}'
source = "Google Finance"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Google Finance](https://www.google.com/finance/quote/NASDAQ:TSLA)

In [41]:
url = f'https://finviz.com/quote.ashx?t={symbol}&p=d'
source = "FinViz"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[FinViz](https://finviz.com/quote.ashx?t=TSLA&p=d)

In [42]:
url = f'https://www.reddit.com/search?q={company}&include_over_18=on&sort=relevance&t=all'
source = "Reddit"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Reddit](https://www.reddit.com/search?q=Tesla&include_over_18=on&sort=relevance&t=all)

In [43]:
morn_exch = morningstarmap[exchange]
url = f'https://www.morningstar.com/stocks/{morn_exch}/{symbol}/quote'
source = "Morningstar"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))
    

[Morningstar](https://www.morningstar.com/stocks/xnas/TSLA/quote)

In [ ]:
url = f'https://whalewisdom.com/stock/{symbol}'
source = "WhaleWisdom"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))


In [ ]:
url = f'https://www.gurufocus.com/stock/{symbol}/guru-trades'
source = "GuruFocus"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))


In [ ]:
https://www.barchart.com/stocks/highs-lows/lows?orderBy=highPercent1y&orderDir=asc
    https://stockcharts.com/h-sc/ui?s=%24DJUSEN&id=p33407302522&def=Y&listNum=1
        https://stockcharts.com/h-sc/ui?s=%24DJUSDN&id=p33407302522&def=Y&listNum=1